<a href="https://colab.research.google.com/github/hanjing06/yolo11s-barbell-detection-model/blob/main/YOLO_Barbell_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### A Guide On How To Train A YOLO Model To Detect Objects (Oriented Bounding Boxes Object Detection).

<small>(Specifically our barbell detection mode)</small>

<br>

**Author**: Hanjing Lin and Mats Leis

**Last Updated**: Friday Feburary 13 2026

<br>


###Introduction:

This notebook uses Ultralytics to train the YOLO11 object detection model using our fixed dataset from the [RoboFlow Universe](https://universe.roboflow.com/techtitans-pcchf/barbell-detection-8phtm/dataset/12). At the end of this Colab, you can run our custom YOLO model that can run on your PC, phone, or edge device like a Raspberry Pi

### Working in Colab
Colab provides a virtual machine in your browser complete with a Linux OS, filesystem, Python environment, and best of all, a free GPU. We'll install PyTorch and Ultralytics in this environment and use it to train our model. Simply click the Play button on sections of code in this notebook to execute them on the virtual machine.

###Verifying the GPU availability

Make sure you're using a GPU-equipped machine by going to "Runtime" -> "Change runtime type" in the top menu bar, and then selecting one of the GPU options (I'd recomment T4 GPU) in the Hardware accelerator section. **Click Play on the following code block below to verify that the NVIDIA GPU is present and ready for training.**

<p align=center>
<img src="https://raw.githubusercontent.com/hanjing06/yolo11s-barbell-detection-model/refs/heads/main/docs/Runtime_Config.png" height="360"><br>
<i>Figure 1: Recommended settings for runtime.</i>
</p>



In [ ]:
!nvidia-smi

Fri Feb 13 07:32:12 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

###1. Training our model

*Note that we have collected an already annotated and labeled dataset with over 500+ images from Roboflow*

*However, if you'd like to gather and label your own images, check the note at the end*

To start, make sure that your dataset folder has the same structure has Figure 2, and that you have a `classes.txt` file that contains all the classes

<p align=center>
<img src="https://raw.githubusercontent.com/hanjing06/yolo11s-barbell-detection-model/refs/heads/main/docs/Folder_Structure.png" height="300"><br>
<i>Figure 2: Folder structure, you can
<a href="https://github.com/hanjing06/yolo11s-barbell-detection-model/blob/3e668770eba20b934fb2c786e923721aa748acb7/docs/barbellData.zip" download="barbellData.zip">download the zip</a> from the GitHub as an example.</i>
</p><br>

Once you've got your dataset built, put into the file structure shown above, and zipped into `barbellData.zip`, you're ready to move on to the next step.

###2. Training the model

Next, we'll upload our dataset (2.1) and prepare it for training with YOLO. We'll split the dataset into train and validation folders (2.2), and we'll automatically generate the configuration file for training the model.

*Note: If your Roboflow dataset already came with train and validation folders, you can skip 2.2.*

###2.1 Uploading the zip to Colab

We now have to upload the `barbellData.zip` file into Colab as shown in Figure 3.

<p align=center><img src="https://raw.githubusercontent.com/hanjing06/yolo11s-barbell-detection-model/refs/heads/main/docs/Upload_Colab_Zip.png"><i>Figure 3: How to upload a zip to Colab</p>

###2.2 Split images into testing and validation folders

Next line unzips the data files, and creates a folder for it. Play the code below to extract.

*Note: for personal use/modifications, you can change the file name to match the one you're using*

In [ ]:
!unzip -q /content/barbellData.zip -d /content/barbellData

Ultralytics requires a particular folder structure to store training data for models. You have your root folder, then two main folders inside:

**Train:** Actual images to train the model. They train in epochs. One epoch of training means every image in the train set is passed into the <a href="https://www.geeksforgeeks.org/deep-learning/neural-networks-a-beginners-guide/">neural network</a>. The training algorithm adjusts the network weights to fit the data in the images.

**Validation:** Images are used to check model's performance at the end of each training epoch.

*Note that epoch stands for how long the model trains for. For example: 60 epochs, it'll train with 60 images first.*

It is recommended that for a dataset with less than 200 images, 60 epochs should be good, more than 200, 40 is good

*Another note: you can change the model in the line below to adjust your training size/accuracy*

Run the python code below made by <a href="https://www.linkedin.com/in/evan-juras/">Evan Juras</a> to create a required folder of 90% of the images going into a training folder, and 10% of the images in the validations folder. (These folders will be in the root called "data")

In [ ]:
!wget -O /content/train_val_split.py https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/utils/train_val_split.py
!python train_val_split.py --datapath="/content/barbellData" --train_pct=0.9

--2026-02-13 17:49:05--  https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/utils/train_val_split.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3203 (3.1K) [text/plain]
Saving to: ‘/content/train_val_split.py’

/content/train_val_ 100%[===================>]   3.13K  --.-KB/s    in 0s      

2026-02-13 17:49:05 (62.1 MB/s) - ‘/content/train_val_split.py’ saved [3203/3203]

Number of image files: 262
Number of annotation files: 262
Images moving to train: 235
Images moving to validation: 27


###3. Installing Ultralytics

Next, we'll install the Ultralytics library in this Google Colab instance. This Python library will be used to train the YOLO model.

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.3 MB/s eta 0:00:00


###4. Configuring the training (Pre-training)

Last step before running the training. We need to create a YAML file. This file specifies the location of your train and validation data, and it also defines the model's classes. An example configuration file model is available [here](https://github.com/ultralytics/ultralytics/blob/main/ultralytics/cfg/datasets/coco128.yaml).

To generate a YAML file. Run the python script below.

In [ ]:
# Python function to automatically create data.yaml config file
# 1. Reads "classes.txt" file to get list of class names
# 2. Creates data dictionary with correct paths to folders, number of classes, and names of classes
# 3. Writes data in YAML format to data.yaml

import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': '/content/data',
      'train': 'train/images',
      'val': 'validation/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = '/content/barbellData/classes.txt'
path_to_data_yaml = '/content/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/data.yaml

Created config file at /content/data.yaml

File contents:

path: /content/data
train: train/images
val: validation/images
nc: 3
names:
- barbell
- plate
- person


###5. Training the model

####5.1 Parameters

We are training the model on Colab which uses a cloud-based GPU but you can nonetheless <a href="https://docs.ultralytics.com/modes/train/">train locally.</a>

<br>Below are some learnings about machine learning (yay!)

####5.1.1 Choosing a model

<p align=center><a href="https://docs.ultralytics.com/compare/#interactive-performance-benchmarks"><img src="https://cdn.prod.website-files.com/680a070c3b99253410dd3df5/684d853f09317da8ad90cd28_67ed50e66edd3138c81f2636_67dd6e628e9151bd18fdb88a_rcnn_fig6.webp"></a><i>Figure 4: Performance Benchmarks. <a href="https://docs.ultralytics.com/compare/#quick-decision-guide">Check out their quick decision guide here.</a></i></p><br>

We are using the `yolo11s.pt` model

(Recall) Number of epochs is number of "pass throughs", it correlates to how long the model will train for. A good rule of thumb is: > 200 images = 60 epochs, < 200 images = 40 epochs

Resolution of the image has a large impact as it affects speed and accuracy. Higher res = slower but more accurate, lower res = faster but less accurateYOLO models are typically trained and inferenced at a 640x640 resolution. However, if you want your model to run faster or know you will be working with low-resolution images, try using a lower resolution like 480x480. If you wanna learn why this is a case, I'd recommend this video at 2:45: <a href="https://youtu.be/aircAruvnKk?si=jBqO1Ce5bRHGaXno&t=165">
But what is a neural network? | Deep learning chapter 1</a> By 3Blue1Brown

###5.2 Run the training

Run the following code block to begin training. If you want to use a different model, number of epochs, or resolution, change model, epochs, or imgsz.

The default will be the `yolo11s.pt` model at `60 epochs` with a `640x640` resolution

In [ ]:
!yolo detect train data=/content/data.yaml model=yolo11s.pt epochs=60 imgsz=640

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots

What the console is doing is:
1. parsing the images in the training and validation directories.
2. training the model.
3. during each epoch cycle, runs the model on the validation dataset and reports the resulting <a href="https://jonathan-hui.medium.com/map-mean-average-precision-for-object-detection-45c121a31173">mAP</a>, precision, and recall.
4. the program will end once the number of epochs is reached (default case: 60 epochs)

Then, once the training is complete, the best trained model weights will be saved in `content/runs/detect/train/weights/best.pt`. Information about training in the `content/runs/detect/train` will have a `results.png` this file will demonstrate how loss, precision, recall, and mAP progressed over each epoch.

###6. The Test Model

Now the model has been trained, let's test it! Run the command below, it'll run the model on the images in the validation folder and then display the results for the first 10 images. This is a good way to confirm your model is working as expected. Click Play on the blocks below to see how your model performs.

In [ ]:
!yolo detect predict model=runs/detect/train/weights/best.pt source=data/validation/images save=True

In [ ]:
import glob
from IPython.display import Image, display
for image_path in glob.glob(f'/content/runs/detect/predict/*.jpg')[:10]:
  display(Image(filename=image_path, height=400))
  print('\n')

The model should be drawing boxes, the same thing you would've done in Label Studio. If it is not doing a great job, you might need to increase the epochs, use a larger model, add more images to the training dataset or double check there is no labelling errors.


###7. Deplying the model (How to download your newly trained model and run it on a local device)

Now that your custom model has been trained, it's ready to be downloaded and deployed in an application! YOLO models can run on a wide variety of hardware, including PCs, embedded systems, and phones. Ultralytics makes it easy to convert the YOLO models to various formats (`tflite`, `onnx`, etc.) and deploy them in a variety of environments.

###7.1 Download YOLO Model

1. Zip and download the trained model by running the code block below.
2. It'll create a folder called `barbell_detection_model`.
3. Moves the model weights into it.
4. Renames `best.pt` to `barbell_detection_model.pt`.
5. Adds the training results to reference for later.
6. Zips the whole thing.

In [ ]:
# Create "model_name" folder to store model weights and train results
!mkdir /content/barbell_detection_model
!cp /content/runs/detect/train/weights/best.pt /content/barbell_detection_model/barbell_detection_model.pt
!cp -r /content/runs/detect/train /content/barbell_detection_model

# Zip into "the .zip file"
%cd barbell_detection_model
!zip /content/barbell_detection_model.zip barbell_detection_model.pt
!zip -r /content/barbell_detection_model.zip train
%cd /content

###7.2 Run the model locally

For python on your local device, <a href="https://www.linkedin.com/in/evan-juras/">Evan Juras</a> wrote this <a href="https://github.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/blob/main/yolo_detect.py">python script</a> that loads the model, run inference on an image, parse the inference results, and displays boxes around each detected class in the image.

You can download using https://www.anaconda.com/download but for this, we are just going to `pip install`. Below are simple steps to setting up a project directory and setitng up a python environment in the bash terminal.

<table><tr><td width="50%">

**macOS/Linux**
```bash
mkdir project_name
cd project_name
python -m venv venv
source venv/bin/activate
```

</td>
<td width="50%">

**Windows**
```bash
mkdir project_name
cd project_name
python -m venv venv
venv\Scripts\activate
```

</td></tr></table>


1. Install Ultralytics using the terminal

```bash
  pip install -U ultralytics
```

2.Download the YOLO model script and put it into the project root

      curl -o yolo-detector.py /path/to/projectroot

To run inference with a yolov8s model on a USB camera at 1280x720 resolution, issue:

      python yolo_detect.py --model barbell_detection_model.pt --source usb0 --resolution 1280x720

If you are having trouble setting up your IDE, are missing packages, or want to install using Conda or Docker, instead of pip, refer to the Ultralytics <a href="https://docs.ultralytics.com/quickstart/#conda-docker-image">intallation guide</a>.

